In [77]:
import pandas as pd
import sqlite3
import os
import re

In [2]:
directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Tools

In [113]:
def save_db(db_name, df_name, *path):
    if len(path) == 1:
        conn_save = sqlite3.connect(directory + path[0] + db_name + '.db')
    else:
        conn_save = sqlite3.connect(directory + '/dataPrep/source/' + db_name + '.db')
    df_name.to_sql(name=db_name, con=conn_save, if_exists='replace', index=False)

In [80]:
def pid_count(df, *the_rest):
    def _cal(df):
        return len(df.drop_duplicates('pid'))
    print(*list(map(_cal, [df]+list(the_rest))), sep=', ')

In [94]:
def link(pid):
    return print(all_date.loc[all_date.pid==str(pid)].proj_url[:1].item())

In [156]:
def describe_table(df):
    table = df.groupby('proj_category').mean()[['proj_backer_count']].rename(
    columns={'proj_backer_count': 'Mean'}).reset_index().merge(
    df.groupby('proj_category').min()[['proj_backer_count']].rename(
        columns={'proj_backer_count': 'Min'}).reset_index(), 
    on=['proj_category']).merge(
    df.groupby('proj_category').quantile(.25)[['proj_backer_count']].rename(
        columns={'proj_backer_count': 'Q1'}).reset_index(), 
    on=['proj_category']).merge(
    df.groupby('proj_category').quantile(.5)[['proj_backer_count']].rename(
        columns={'proj_backer_count': 'Q2'}).reset_index(), 
    on=['proj_category']).merge(
    df.groupby('proj_category').quantile(.75)[['proj_backer_count']].rename(
        columns={'proj_backer_count': 'Q3'}).reset_index(),
    on=['proj_category']).merge(    
    df.groupby('proj_category').max()[['proj_backer_count']].rename(
        columns={'proj_backer_count': 'Max'}).reset_index(), 
    on=['proj_category']).merge(
    df.groupby('proj_category').count()[['pid']].rename(
        columns={'pid': 'Total'}).reset_index(), on=['proj_category'])
    return table

# Data

##### produced order
1. proj_urls
2. date_fund, date_rew
3. all_date, all_date_year
4. proj, rew, faq, upd, cmt (ds_ind_26229)
5. first_last_date, dates
6. _all, _year, _pre (ds_ind_26229)
7. cmt_rmO_lm_b03
8. rmO_lm_b03, rmO_lm_b03_neg
9. proj_factor, crt_exp, interact_faq_upd, interact_cmt

In [100]:
pid_count(date_fund, date_rew, all_proj, all_date)

127433, 127366, 127433, 127433


## 1. & 2. Explore & Time-related

In [19]:
"""
127,440 projects funded projects before July 8 2017.
Our dataset has 120,825 URLs crawled from the Explore page.
Finally we have 127,433 URLs after merge with an external dataset provided by CrowdBerkeley.
"""
conn_proj_urls = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'proj_urls.db')

# time-related data ... funding& reward
conn_date_fund = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'date_fund.db')
conn_date_rew = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'date_rew.db')

In [20]:
proj_urls = pd.read_sql_query('SELECT * FROM proj_urls', conn_proj_urls)
date_fund = pd.read_sql_query('SELECT * FROM date_funding', conn_date_fund)
date_rew = pd.read_sql_query('SELECT * FROM date_reward', conn_date_rew)

In [181]:
pid_count(proj_urls, date_fund, date_rew)

127433, 127433, 127366


In [75]:
# we had merged with funding dates and category.
proj_urls.head(1)

,pid,proj_end_date,proj_start_date,proj_url,category,subcategory
0,841825846,2012-08-25,2012-07-26,https://www.kickstarter.com/projects/834182258...,food,Food


In [27]:
date_fund.head(1)

,pid,proj_end_date,proj_start_date,proj_url
0,841825846,2012-08-25T15:35:39-04:00,2012-07-26T15:35:39-04:00,https://www.kickstarter.com/projects/834182258...


In [82]:
date_rew.head(1)

,pid,rew_amount_required,rew_backer_count,rew_backer_limit,rew_delivery,rew_id
0,841825846,1.0,5.0,None,2012-10-01,0.0


## 3. create a dataset contains all time-related data

In [87]:
# Merge categories into 'date_rew' dataframe
all_date = date_rew.merge(all_proj[['pid','proj_start_date','proj_end_date',
                                    'category','subcategory','proj_url']], on=['pid'], how='outer').copy()
# save_db('all_date', all_date)

In [133]:
# conn_all_date = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'all_date.db')
# all_date = pd.read_sql_query("SELECT * FROM all_date", conn_all_date)

#### projects which deliver rewards within 2015-07 and 2016-06

In [102]:
all_date_year = all_date[(all_date['rew_delivery'] > '2015-06-30') & 
                         (all_date['rew_delivery'] < '2016-07-01')].sort_values('category')
# save_db('all_date_year', all_date_year)

In [104]:
pid_count(all_date_year)

26229


In [185]:
all_date_year.head(1)

,pid,rew_amount_required,rew_backer_count,rew_backer_limit,rew_delivery,rew_id,proj_start_date,proj_end_date,category,subcategory,proj_url
292008,1446685548,2000.0,0.0,Limited,2015-12-01,10.0,2015-07-22,2015-08-31,art,Painting,https://www.kickstarter.com/projects/colleenbr...


In [131]:
# conn_all_date_year = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'all_date_year.db')
# all_date_year = pd.read_sql_query("SELECT * FROM all_date_year", conn_all_date_year)

## 4. Projects, Rewards, FAQs, Updates, and Comments

In [141]:
# conn_proj = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'proj.db')
# proj = pd.read_sql_query("SELECT * FROM proj", conn_proj)

# conn_rew = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'rew.db')
# rew = pd.read_sql_query("SELECT * FROM rew", conn_rew)

# conn_faq = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'faq.db')
# faq = pd.read_sql_query("SELECT * FROM faq", conn_faq)

# conn_upd = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'upd.db')
# upd = pd.read_sql_query("SELECT * FROM upd", conn_upd)

# conn_cmt = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'cmt.db')
# cmt = pd.read_sql_query("SELECT * FROM cmt", conn_cmt)

In [159]:
describe_table(proj)

,proj_category,Mean,Min,Q1,Q2,Q3,Max,Total
0,art,106.030194,1.0,22.00,45.0,98.00,7663.0,1954
1,comics,230.397093,1.0,47.00,98.0,215.00,7119.0,1307
2,crafts,91.062837,1.0,17.00,38.0,93.00,1203.0,557
3,dance,70.880223,1.0,28.00,48.0,85.00,863.0,359
4,design,631.510239,1.0,65.00,163.0,455.75,78471.0,2344
5,fashion,240.359836,1.0,38.00,89.0,191.00,13642.0,1220
6,film & video,209.717241,1.0,27.00,66.0,163.00,48270.0,4205
7,food,222.149442,1.0,45.00,103.0,200.00,13116.0,1345
8,games,846.469849,1.0,84.00,227.5,635.00,219382.0,2786
9,journalism,157.563433,1.0,27.75,66.0,185.75,1951.0,268


# 5. the last delivery day and the day after one year

In [108]:
from dateutil.relativedelta import relativedelta

# 2015-07-01 to 2016-06-31
first_day = datetime.datetime.strptime('2015-07-01', "%Y-%m-%d")
last_day = datetime.datetime.strptime('2016-06-30', "%Y-%m-%d")

# choose the last EDD
rew['rew_delivery'] = rew['rew_delivery'].apply(
    lambda str_date: datetime.datetime.strptime(str_date, "%Y-%m-%d"))
rew_in_year = rew[(rew.rew_delivery >= first_day) & (rew.rew_delivery <= last_day)]
rew_in_year = rew_in_year.sort_values('rew_delivery')

# we choose the last delivery date of each project 
first_last_date = rew_in_year.drop_duplicates('pid', keep='last') 

# set the date after the last EDD
first_last_date = first_last_date[['pid', 'rew_delivery']].copy()
first_last_date = first_last_date.rename(index=str, columns={'rew_delivery':'first_date'})

one_year = relativedelta(months=12)
first_last_date['last_date'] = first_last_date.first_date.apply(lambda ori_date: ori_date + one_year)

# save_db('first_last_date', first_last_date)

In [132]:
# conn_first_last_date = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'first_last_date.db')
# first_last_date = pd.read_sql_query("SELECT * FROM first_last_date", conn_first_last_date)

In [203]:
first_last_date.head(1)

,pid,first_date,last_date
237546,855332797,2015-07-01,2016-07-01


In [201]:
# processed in 5. Interaction in FAQs and Updates
conn_dates = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'dates.db')
dates = pd.read_sql_query("SELECT * FROM dates", conn_dates)

In [202]:
dates.head(1)

,pid,proj_start_date,first_date,after_d_3,after_d_6,after_d_9,after_d_1,interval_0,interval_3,interval_6,interval_9,interval_1,interval,interval_0103,interval_0406,interval_0709,interval_1012,interval_0106,interval_0712
0,1446685548,2015-07-22 00:00:00,2015-12-01 00:00:00,2016-03-01 00:00:00,2016-06-01 00:00:00,2016-09-01 00:00:00,2016-12-01 00:00:00,132.0,91.0,183.0,275.0,366.0,498.0,91.0,92.0,92.0,366.0,183.0,183.0


In [204]:
len(dates)

26229

# 6. divide data into 'before the last EDD' and 'within a year after the day'

In [160]:
# conn_cmt_all = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'cmt_all.db')
# conn_cmt_year = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'cmt_year.db')
# conn_cmt_pre = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'cmt_pre.db')

# cmt_all = pd.read_sql_query("SELECT * FROM cmt_all", conn_cmt_all)
# cmt_year = pd.read_sql_query("SELECT * FROM cmt_year", conn_cmt_year)
# cmt_pre = pd.read_sql_query("SELECT * FROM cmt_pre", conn_cmt_pre)

# conn_faq_all = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'faq_all.db')
# conn_faq_year = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'faq_year.db')
# conn_faq_pre = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'faq_pre.db')

# faq_all = pd.read_sql_query("SELECT * FROM faq_all", conn_faq_all)
# faq_year = pd.read_sql_query("SELECT * FROM faq_year", conn_faq_year)
# faq_pre = pd.read_sql_query("SELECT * FROM faq_pre", conn_faq_pre)

# conn_upd_all = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'upd_all.db')
# conn_upd_year = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'upd_year.db')
# conn_upd_pre = sqlite3.connect(directory + '/dataPrep/source/ds_ind_26229/' + 'upd_pre.db')

# upd_all = pd.read_sql_query("SELECT * FROM upd_all", conn_upd_all)
# upd_year = pd.read_sql_query("SELECT * FROM upd_year", conn_upd_year)
# upd_pre = pd.read_sql_query("SELECT * FROM upd_pre", conn_upd_pre)

## Comments

In [112]:
cmt.cmt_datetime = cmt.cmt_datetime.apply(lambda date: str(date))
cmt.cmt_datetime = cmt.cmt_datetime.apply(lambda str_date: re.sub('\"', '',str_date))
cmt.cmt_datetime = cmt.cmt_datetime.apply(lambda str_date: re.sub('-.[\d]:.[\d]$', '',str_date))
cmt.cmt_datetime = cmt.cmt_datetime.apply(lambda str_date: re.sub('T', ' ',str_date))
cmt.cmt_datetime = cmt.cmt_datetime.apply(
    lambda str_date: datetime.datetime.strptime(str_date, "%Y-%m-%d %H:%M:%S"))

cmt_and_f_l_date = cmt.merge(first_last_date[['pid','first_date', 'last_date']], on=['pid']).copy()

In [115]:
# Keep only comments (cmt_all) 'before' one year after the first delivery date
cmt_all = cmt_and_f_l_date[cmt_and_f_l_date.cmt_datetime < cmt_and_f_l_date.last_date].copy()
cmt_all = cmt_all[cmt_and_f_l_date.columns.values.tolist()[:-2]]

# Keep only comments (cmt_year) 'within' the last delivery date and one year after the last delivery date
cmt_year = cmt_and_f_l_date[(cmt_and_f_l_date.cmt_datetime >= cmt_and_f_l_date.first_date) & 
                      (cmt_and_f_l_date.cmt_datetime < cmt_and_f_l_date.last_date)].copy()
cmt_year = cmt_year[cmt_and_f_l_date.columns.values.tolist()[:-2]]

# Keep only comments (cmt_pre) 'before' the last delivery date
cmt_pre = cmt_and_f_l_date[cmt_and_f_l_date.cmt_datetime < cmt_and_f_l_date.first_date].copy()
cmt_pre = cmt_pre[cmt_and_f_l_date.columns.values.tolist()[:-2]]

## FAQs

In [118]:
faq.faq_date = faq.faq_date.apply(lambda str_date: re.sub('\"', '',str_date))
faq.faq_date = faq.faq_date.apply(lambda str_date: re.sub('-.[\d]:.[\d]$', '',str_date))
faq.faq_date = faq.faq_date.apply(lambda str_date: datetime.datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S"))

faq.pid = faq.pid.apply(lambda f_pid: str(int(f_pid)))
faq_and_f_l_date = faq.merge(first_last_date[['pid', 'first_date', 'last_date']], on=['pid']).copy()

faq_all = faq_and_f_l_date[faq_and_f_l_date.faq_date < faq_and_f_l_date.last_date].copy()
faq_all = faq_all[faq_and_f_l_date.columns.values.tolist()[:-2]]

faq_year = faq_and_f_l_date[(faq_and_f_l_date.faq_date >= faq_and_f_l_date.first_date) & 
                            (faq_and_f_l_date.faq_date < faq_and_f_l_date.last_date)].copy()
faq_year = faq_year[faq_and_f_l_date.columns.values.tolist()[:-2]]

faq_pre = faq_and_f_l_date[faq_and_f_l_date.faq_date < faq_and_f_l_date.first_date].copy()
faq_pre = faq_pre[faq_and_f_l_date.columns.values.tolist()[:-2]]

## Updates

In [122]:
upd.upd_date = upd.upd_date.apply(lambda str_date: re.sub('\"', '',str_date))
upd.upd_date = upd.upd_date.apply(lambda str_date: re.sub('-.[\d]:.[\d]$', '',str_date))
upd.upd_date = upd.upd_date.apply(lambda str_date: datetime.datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S"))

upd.pid = upd.pid.apply(lambda f_pid: str(int(f_pid)))
upd_and_f_l_date = upd.merge(first_last_date[['pid', 'first_date', 'last_date']], on=['pid']).copy()

upd_all = upd_and_f_l_date[upd_and_f_l_date.upd_date < upd_and_f_l_date.last_date].copy()
upd_all = upd_all[upd_and_f_l_date.columns.values.tolist()[:-2]]

upd_year = upd_and_f_l_date[(upd_and_f_l_date.upd_date >= upd_and_f_l_date.first_date) & 
                            (upd_and_f_l_date.upd_date < upd_and_f_l_date.last_date)].copy()
upd_year = upd_year[upd_and_f_l_date.columns.values.tolist()[:-2]]

upd_pre = upd_and_f_l_date[upd_and_f_l_date.upd_date < upd_and_f_l_date.first_date].copy()
upd_pre = upd_pre[upd_and_f_l_date.columns.values.tolist()[:-2]]

# 7. Eliminate Outliers and Inadequate Data

> processed in 2. Descriptive Statistics in Different Date Range

In [134]:
conn_cmt_rmO_lm_b03 = sqlite3.connect(directory + '/dataPrep/source/overall/' + 'cmt_rmO_lm_b03.db')
cmt_rmO_lm_b03 = pd.read_sql_query("SELECT * FROM cmt_rmO_lm_b03", conn_cmt_rmO_lm_b03)

In [135]:
len(cmt_rmO_lm_b03)

2137

In [157]:
describe_table(cmt_rmO_lm_b03)

,proj_category,Mean,Min,Q1,Q2,Q3,Max,Total
0,art,106.117647,54.0,69.00,96.0,138.00,211.0,85
1,comics,235.163462,121.0,149.75,227.0,305.25,466.0,104
2,crafts,95.465517,50.0,61.00,80.5,131.00,202.0,58
3,dance,90.000000,64.0,77.00,90.0,103.00,116.0,2
4,design,477.584178,217.0,283.00,412.0,641.00,1041.0,493
5,fashion,204.061224,104.0,137.50,179.0,259.50,406.0,147
6,film%20&%20video,198.596491,87.0,119.00,183.0,260.00,352.0,57
7,food,239.833333,122.0,156.00,212.5,324.25,406.0,48
8,games,670.891844,310.0,425.50,598.0,870.00,1457.0,564
9,journalism,228.333333,107.0,167.50,228.0,289.00,350.0,3


# 8. Failed Projects
> processed in 3. Failed Project

In [197]:
conn_rmO_lm_b03 = sqlite3.connect(directory + '/dataPrep/source/constructs/' + 'rmO_lm_b03.db')
rmO_lm_b03 = pd.read_sql_query("SELECT * FROM rmO_lm_b03", conn_rmO_lm_b03)

In [199]:
len(rmO_lm_b03)

2137

In [153]:
conn_rmO_lm_b03_neg = sqlite3.connect(directory + '/dataPrep/source/constructs/' + 'rmO_lm_b03_neg.db')
rmO_lm_b03_neg = pd.read_sql_query("SELECT * FROM rmO_lm_b03_neg", conn_rmO_lm_b03_neg)

In [155]:
describe_table(neg_rmO_lm_b03)

,proj_category,Mean,Min,Q1,Q2,Q3,Max,neg_rmO_lm_b03
0,art,108.894737,61.0,79.00,97.0,131.00,191.0,19
1,comics,216.842105,126.0,138.00,177.0,258.00,466.0,19
2,crafts,105.375000,57.0,60.50,90.5,135.50,201.0,16
3,design,479.542453,217.0,273.50,412.0,634.50,1041.0,212
4,fashion,208.557377,104.0,133.00,182.0,272.00,406.0,61
5,film & video,229.250000,115.0,142.25,256.0,304.50,318.0,8
6,food,240.500000,127.0,160.00,212.5,323.75,377.0,10
7,games,703.242038,315.0,458.00,629.0,909.00,1398.0,157
8,music,111.727273,74.0,80.50,105.0,132.00,182.0,11
9,photography,160.857143,98.0,122.50,133.0,185.00,280.0,7


In [158]:
len(neg_rmO_lm_b03)

762

# 9. Constructs

In [192]:
conn_proj_factor = sqlite3.connect(directory + '/dataPrep/source/constructs/' + 'proj_factor.db')
proj_factor = pd.read_sql_query("SELECT * FROM proj_factor", conn_proj_factor)

In [183]:
conn_crt_exp = sqlite3.connect(directory + '/dataPrep/source/constructs/' + 'crt_exp.db')
crt_exp = pd.read_sql_query("SELECT * FROM crt_exp", conn_crt_exp)

In [193]:
conn_interact_faq_upd = sqlite3.connect(directory + '/dataPrep/source/constructs/' + 'interact_faq_upd.db')
interact_faq_upd = pd.read_sql_query("SELECT * FROM interact_faq_upd", conn_interact_faq_upd)

In [194]:
conn_interact_cmt = sqlite3.connect(directory + '/dataPrep/source/constructs/' + 'interact_cmt.db')
interact_cmt = pd.read_sql_query("SELECT * FROM interact_cmt", conn_interact_cmt)

In [195]:
pid_count(proj_factor, crt_exp, interact_faq_upd, interact_cmt)

2137, 2137, 2137, 2137
